In [2]:
from typing import List

Identify palindrome:
- Check if it equals its inverse
- Two pointers. Starting from leftmost and rightmost, converge towards middle.
- Two pointers. Starting at `middle` and `middle+1` (for even) AND both starting at `middle` (for odd)
    - **Best**: reusing the result for inner palindrome for outer palindrome candidate. 
Problems:
- Longest palindrome substring
- Longest palindrome subsequence
- Find All palindromic substring
- *Longest common subsequence of two strings*
- 

Expanding Approach: Loop through all i. Expand from (i,i) and (i,i+1) using two pointers. 
- longest palindrome substring
- Find all palindromic substring


In [7]:
# Longest palindrome substring
def longestPalindrome(s: str) -> str:
    '''
    Expand from the middle -> both sides outwards

    abba
     ^
    ^ ^
   ^   ^
  -1   len(s)
    '''
    def expand(lo, hi):
        while lo >= 0 and hi < len(s) and s[lo] == s[hi]:
            lo -= 1
            hi += 1
        return s[lo+1: hi]

    longest = ""
    for i in range(len(s)):
        odd_palindrome = expand(i,i)
        even_palindrome = expand(i,i+1)
        longest = max(odd_palindrome, even_palindrome, longest, key=len)

    return longest
    
   

In [8]:
# Find all palindromic substring
def countSubstrings(s: str) -> int:
    '''
    abba
    ^^^^

    '''
    N = len(s)
    count = 0   # number of all palindrome substrings

    def expand(left, right):
        while left >= 0 and right < N and s[left] == s[right]:
            nonlocal count
            count += 1
            left -= 1
            right += 1


    for middle in range(N):
        # Look for all odd palindromes centered here
        expand(middle, middle)

        # Look for all even palindromes centered here
        expand(middle, middle+1)

    return self.count

    '''
    Time: O(N^2)
    Space: O(1)
    '''

Approach: DP
- Longest palindrome subsequence || is Valid K-palindrome
- Longest common subsequence 

In [10]:
# K-palindrome: can be a palindrome by removing at most `k` chars from it
# Equivalent to finding if the Longest Palindromic Subsequence hash length >= N-k
def isValidKPalindrome(s: str, k: int) -> bool:
    '''
    <=> Find any palindromic subsequence of length at least N-K    => #516
    (where N is the length of the string)

    * All intervals below are inclusive at both ends (contrary to python's default)
    If we know len(longest palindrome subseq of s[i+1 : j-1])
                  abbacda   
                   ^   ^
                 i+1   j-1 

    then it helps finding len(longest palindrome subseq of s[i:j])
                  abbacdb   
                  ^     ^
                  i     j

        - If s[i] == s[j], then this "margin pair" can be added (+2)
        - Else, these may be helpful:
            - len(longest palindrome subseq of s[i+1 : j]) 
            - len(longest palindrome subseq of s[i : j-1])
            - get the max. 

    For each (i,j), I need to first know its left, down, down-left neighbors

      0 1 2 3 4
    0 1 ? X Y
    1   1 ? X Y
    2     1 ? X
    3       1 ?
    4         1

    ''' 
    longest = 1     # longest subseq length
    N = len(s)
    dp = [[0]*N for _ in range(N)]

    # Initialization: set main diagonal, and the next diagonal
    for i in range(N):
        dp[i][i] = 1
    for i in range(N-1):
        if s[i] == s[i+1]:
            dp[i][i+1] = 2
            longest = 2
        else:
            dp[i][i+1] = 1

    # Main Loop.
    for shift in range(2, N):
        for i in range(0, N - shift):
            j = i + shift

            # Use solutions to subproblems to solve the current problem 
            # 1. Can use the margin pair
            if s[i] == s[j]:
                dp[i][j] = dp[i+1][j-1] + 2
                longest = max(longest, dp[i][j])

            # 2. Can't use. ...
            else:
                dp[i][j] = max(dp[i+1][j], dp[i][j-1])

    # See if the longest palindrome subseq hasn't removed too much chars from the original.
    return longest >= N-k


In [ ]:
# Longest Common Subsequence
# Very useful in `diff`ing files, DNA sequences, ...
class LongestCommonSubsequence:
    def longestCommonSubsequence(self, text1: str, text2: str) -> int:
        '''
        DP. 
          a b c d e
        a 3 2 2 1 1
        c 2 2 2 1 1
        e 1 1 1 1 1  0utside
                    0
        
        
        To know dp[row][col], 
        - If text1[row] matches text2[col], then this cell is 1 + dp[row+1][col+1]
        - Otherwise, put the max of dp[row][col+1] and dp[row+1][col]
        
        Boundaries ... used all chars of a word => 0
        Build the dp table bottom up, from right to left.
        '''
        ROWS, COLS = len(text1), len(text2)
        dp = [[0]*COLS for _ in range(ROWS)]
        
        def inbound(row, col):
            return (row < ROWS) and (col < COLS)
        
        for col in range(COLS-1, -1, -1):
            for row in range(ROWS-1, -1, -1):
                # 1. current chars match
                if text1[row] == text2[col]:
                    dp[row][col] = 1 + (dp[row+1][col+1] if inbound(row+1,col+1) else 0)
                else:
                    # 2. don't match. Exclude the one that'd lead to a shorter common subseq
                    right = dp[row][col+1] if inbound(row,col+1) else 0
                    down = dp[row+1][col] if inbound(row+1,col) else 0
                    dp[row][col] = max(right, down)
                    
        return dp[0][0]
        
        
        
    
    def longestCommonSubsequence_Recursive(self, text1: str, text2: str) -> int:
        '''
        [**] DP (text1, text2)
        - If text1[0] and text2[0] matches, then there's no reason not to include them.
            - Then can move on to DP(text1[1:], text2[1:])
        - Elif the first chars differ, then they must not both exist in the LCS
            - At most 1 of them can exist in the LCS. 
            - Try both cases. 
            - DP(text1[1:], text2)
            - DP(text1, text2[1:])
        '''
        
        def getLCS(index1, index2):
            if index1 == len(text1) or index2 == len(text2):
                return 0
            
            # 1. First chars match. No reason not to include. 
            # Must contribute to the length of LCS
            if text1[index1] == text2[index2]:
                return 1 + getLCS(index1+1, index2+1)
            else:
                # 2. First chars differ. Cannot simultaneously contribute to LCS.    
                len_discard2 = getLCS(index1, index2+1)
                len_discard1 = getLCS(index1+1, index2)
                return max(len_discard2, len_discard1)
        
        return getLCS(0, 0)


# Binary search
### Capacity To Ship Packages Within D Days
    - BinSearch among the maxCapacity and minCapacity.
    - maxCapacity = sum of all weights
    - minCapacity = weight of the heaviest item


### Find Minimum in Rotated Sorted Array

In [6]:
'''
Find Minimum in Rotated Sorted Array. 
'''
def findMin(nums: List[int]) -> int:
    # If the list has just one element then return that element.
    if len(nums) == 1:
        return nums[0]

    left = 0
    right = len(nums) - 1

    # if the last element is greater than the first element then there is no rotation.
    # e.g. 1 < 2 < 3 < 4 < 5 < 7. Already sorted array.
    # Hence the smallest element is first element. A[0]
    if nums[right] > nums[0]:
        return nums[0]

    # Binary search way
    while right >= left:
        # Find the mid element
        mid = left + (right - left) // 2
        # if the mid element is greater than its next element then mid+1 element is the smallest
        # This point would be the point of change. From higher to lower value.
        if nums[mid] > nums[mid + 1]:
            return nums[mid + 1]
        # if the mid element is lesser than its previous element then mid element is the smallest
        if nums[mid - 1] > nums[mid]:
            return nums[mid]

        # if the mid elements value is greater than the 0th element this means
        # the least value is still somewhere to the right **as we are still dealing with elements greater than nums[0]**
        if nums[mid] > nums[0]:
            left = mid + 1
        # if nums[0] is greater than the mid value then this means the smallest value is somewhere to the left
        else:
            right = mid - 1
                
findMin([10,11,12,0,1,2,3,4,5,6,7,9])

0

### Median of Two Sorted Arrays

Median = mean of `arr[(N-1)//2]` and `arr[N//2]`    
        1 2 3 4     1 2 3 4 5
        
          ^ ^           ^